# Bài thực hành số 5.1

Chào mừng đến với bài thực hành số 5.1, đây sẽ là phần tiếp diễn của bài thực hành thứ 5 trên colab. Qua bài thực hành này, chúng tôi sẽ giúp bạn biết các áp dụng model vừa được train vào việc nhận dạng vật thể trong video hoặc qua camera laptop.

## Hướng dẫn

Dưới đây là hướng dẫn chi tiết các bước để bạn có thể hiểu được quá trình áp dụng DeepSORT và YOLO lên video.


### Cài đặt thư viện cần thiết

In [21]:
import numpy as np
import torch
import cv2
import math
import time
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

### Tải model nhận diện

Hàm `torch.hub.load` trong PyTorch là một công cụ mạnh mẽ để tải các mô hình hoặc module từ các kho lưu trữ (repositories) đã được chia sẻ trên GitHub. Nó cho phép bạn dễ dàng sử dụng các mô hình đã được huấn luyện sẵn hoặc module khác mà không cần phải tự xây dựng từ đầu.

Cấu trúc của hàm `torch.hub.load`

model = torch.hub.load(repo_or_dir, model, *args, **kwargs)

Tham số của hàm:

1. **`repo_or_dir`**: 
   - Đây là tham số chỉ định kho lưu trữ GitHub chứa mô hình bạn muốn tải. Nó được cung cấp dưới dạng một chuỗi, ví dụ: `'ultralytics/yolov5'`.
   - Bạn cũng có thể chỉ định đường dẫn cục bộ đến thư mục chứa mô hình nếu mô hình đã được tải xuống.

2. **`model`**: 
   - Tên của mô hình hoặc module mà bạn muốn tải từ kho lưu trữ. Ví dụ: `'custom'` cho mô hình tùy chỉnh hoặc `'yolov5s'` cho mô hình YOLOv5s đã được huấn luyện sẵn.
   - Tùy thuộc vào kho lưu trữ, tên này có thể đại diện cho một mô hình cụ thể hoặc một tập hợp các mô hình đã được định nghĩa trước.

3. **`path`**: 
   - Đường dẫn đến file `.pt` chứa trọng số của mô hình đã được huấn luyện. Tham số này thường được sử dụng khi bạn muốn tải một mô hình đã được huấn luyện trước mà bạn tự xây dựng hoặc tùy chỉnh.

4. **`force_reload`**: 
   - Tham số này có giá trị boolean (`True` hoặc `False`).
   - Khi `force_reload=True`, hàm sẽ tải lại mô hình từ kho lưu trữ hoặc đường dẫn được cung cấp ngay cả khi mô hình đã được tải trước đó. Điều này rất hữu ích khi bạn muốn tránh xung đột hoặc đảm bảo rằng bạn đang sử dụng phiên bản mô hình mới nhất.

Bài tập 1: Hoàn thành đoạn code sau bằng việc điền vào [...]

In [ ]:
# force reload: tránh xung đột tham số khi load model mới
model = torch.hub.load('ultralytics/yolov5', [...], path= [...], force_reload = [...])

In [ ]:
# force reload: tránh xung đột tham số khi load model mới
model = torch.hub.load('ultralytics/yolov5', 'custom', path= r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\custom_yolov5_car_v2\weights\best.pt", force_reload = True)

### Test với hình ảnh

Trước khi làm việc với video, chúng ta sẽ thử áp dụng với một vài ảnh để thấy được hiệu quả của model.

**`model(img_path)`**: Dòng lệnh này sử dụng mô hình YOLOv5 đã được tải từ trước để phát hiện các đối tượng trong hình ảnh.
  - **`img_path`** là đường dẫn đến hình ảnh mà bạn muốn chạy phát hiện đối tượng.
  - Khi bạn gọi mô hình với một đường dẫn hình ảnh như thế này, mô hình sẽ xử lý hình ảnh và trả về các kết quả phát hiện dưới dạng một đối tượng `results`.

**`test_detect.show()`**: Phương thức này sẽ hiển thị hình ảnh với các hộp giới hạn (bounding boxes) được vẽ lên các đối tượng đã phát hiện.
  - Các đối tượng phát hiện được hiển thị cùng với các thông tin như nhãn lớp và độ tin cậy của từng đối tượng.


In [ ]:
img_path = [...]

In [ ]:
img = cv2.imread(img_path)

# Hiển thị hình ảnh sau khi đổi kích thước
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
test_detect = model(img_path)
test_detect.show()

### Áp dụng với video

Bài tập 2: Hoàn thành bài tập sau bằng cách điền vào [...]

In [ ]:
# Đọc video đầu vào
input_video_path = [...]  # Đường dẫn đến video đầu vào
output_video_path = [...]  # Đường dẫn đến video đầu ra

# Mở video đầu vào
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Lấy thông số của video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo video writer để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho định dạng MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi khung hình từ BGR sang RGB
    rgb_frame = [...]

    # Dự đoán với YOLOv5
    results = model([...])

    # Lấy kết quả dự đoán dưới dạng DataFrame
    df = results.pandas().xyxy[0]

    # Vẽ các hộp dự đoán lên khung hình
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        label = row['name']
        score = row['confidence']
        
        # Vẽ hộp bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Vẽ nhãn lớp và độ tin cậy
        cv2.putText(frame, f'{label} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Ghi khung hình vào video đầu ra
    out.write(frame)

# Giải phóng tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
# Đọc video đầu vào
input_video_path = r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\Test_video\test_car_2.mp4"  # Đường dẫn đến video đầu vào
output_video_path = r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\Test_video\test_car_2_output.mp4"  # Đường dẫn đến video đầu ra

# Mở video đầu vào
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Lấy thông số của video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo video writer để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho định dạng MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi khung hình từ BGR sang RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Dự đoán với YOLOv5
    results = model(rgb_frame)

    # Lấy kết quả dự đoán dưới dạng DataFrame
    df = results.pandas().xyxy[0]

    # Vẽ các hộp dự đoán lên khung hình
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        label = row['name']
        score = row['confidence']
        
        # Vẽ hộp bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Vẽ nhãn lớp và độ tin cậy
        cv2.putText(frame, f'{label} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Ghi khung hình vào video đầu ra
    out.write(frame)

# Giải phóng tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()
